# Forbes World's Billionaires List 2022 Dashboard
**Authors:**
- Alhasan Gamal
- Omar Atef

**Date: 22 - 04 - 2022**

**In this Script, We will develop a python program that is able to show Forbes World's Billionaires List 2022**

**`AI & MachineLearning intake-2 ITI`**

In [1]:
# Import Libraries
import plotly.express as px
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [2]:
# Creat Dash object
app = Dash(external_stylesheets=[dbc.themes.MINTY, dbc.icons.FONT_AWESOME])
app.title = "Home"

In [3]:
# import datas and show Data
df = pd.read_csv('forbes.csv')
gm = px.data.gapminder()

In [4]:
# show top 3 Billionaires and net worth
names        = list(df['personName'][:3])
net_wort_top = list(df['finalWorth'][:3]/1000)

In [5]:
# Creat cards of top 3 Billionaires
card1 = [
    dbc.CardImg(src="static/images/elon.jpg",top=True),
    dbc.CardBody(
        [
            html.H4(names[0], className="card-title"),
            html.P(
                str(net_wort_top[0]) + " Billions",className="card-text"
            )
        ]
    )
]

card2 = [
    dbc.CardImg(src="static/images/jeff.jpg",top=True),
    dbc.CardBody(
        [
            html.H4(names[1], className="card-title"),
            html.P(
                str(net_wort_top[1]) + " Billions",className="card-text"
            )
        ]
    )
]

card3 = [
    dbc.CardImg(src="static/images/bernard.jpg",top=True),
    dbc.CardBody(
        [
            html.H4(names[2][:15], className="card-title"),
            html.P(
                str(net_wort_top[2]) + " Billions",className="card-text"
            )
        ]
    )
]

In [6]:
# creat birthYear column and count number of Billionaires were born in each year
df2 = df.copy()
df2["birthYear"] = [str(x).split('-')[0] for x in list(df["birthDate"])]
birthyrs = df2["birthYear"].value_counts()[1:]

In [7]:
# creat country DataFrame with iso_alpha and number of Billionaires in each country
my_df = gm[['country','iso_alpha']].drop_duplicates()
my_dict = my_df.set_index('country')['iso_alpha'].to_dict()
df2['iso_alpha'] = df['country'].map(my_dict)
map_df = df2[['country','iso_alpha']]
map_df.columns.tolist()
iso_df = pd.DataFrame(map_df.groupby(map_df.columns.tolist(), as_index=False).size())
iso_df.rename(columns={'size':'individuals'}, inplace=True)

In [8]:
# Creat DropDown to switch type of chart for number of Billionaires in each country
dropdown = html.Div(
    [
        dcc.Dropdown(
            id="dropMenu",
            options=["Map", "BarChart", "Scatter"],
            value="Map",
            clearable=False
        )
    ],style={"width":"60%","padding-left":"33%"}
)

In [9]:
# Creat country_list
country_list = [{"label":str(c), "value":str(c)} for c in df2["country"].unique()]
del country_list[19]

In [10]:
# Creat DropDown to switch country for age histogram
dropdown2 = html.Div(
    [
        dcc.Dropdown(
            id="dropMenu2",
            options= country_list,
            value="",
        )
    ], style={"width":"60%", "padding-left":"33%"}
)

In [11]:
# Creat category DataFrame
df_cat = pd.DataFrame(df.groupby('category', as_index=False)['finalWorth'].sum())
df_bnum = pd.DataFrame(df.groupby('category', as_index=False)['personName'].count())
df_cat['finalWorth'] = df_cat['finalWorth']/1000
df_cat['N_Billionaires'] = df_bnum['personName']

In [12]:
# Creat DropDown yo switch male or female for selfMade or not
dropdown3 = html.Div([
    dcc.Dropdown(
        id="dropMenu3",
        options=[{"label":"Male", "value":"M"}, {"label":"Female", "value":"F"}],
        value=None,
        multi=True
    )
], style={"width":"60%", "padding-left":"33%"})

In [13]:
# Creat Cards of about us
hasan = [
    dbc.CardImg(src="static/images/alhasan.jpg", top=True),
    dbc.CardBody(
        [
            html.H4("Alhasan Gamal", className="card-title"),
            html.P(
                children=[
                    html.Span("LinkedIn: "),
                    html.A("Alhasan Gamal", href='https://www.linkedin.com/in/alhasan-gamal-480473173/')
                ],
                className="card-text"
            )
        ]
    )
]

omar = [
    dbc.CardImg(src="static/images/omar.jpg", top=True),
    dbc.CardBody(
        [
            html.H4("Omar Atef", className="card-title"),
            html.P(
                children=[
                    html.Span("LinkedIn: "),
                    html.A("Omar Gedawy", href='https://www.linkedin.com/in/omar-gedawy/')
                ],
                className="card-text"
            )
        ]
    )
]

In [14]:
# Creat Layout of app
app.layout = html.Div([
    # Creat Simple NavbarSimple
    dbc.NavbarSimple(
        children=[
            dbc.NavLink(
                [html.I(className="fas fa-home me-2"),html.Span("Home")],
                href="/",
                active="exact"
            ),
            dbc.NavLink(html.A("About Us",href="#aboutus"))
        ],
        brand="Forbes World's Billionaires List 2022 Dashboard",
        color="light",
        dark= False
    ),
    dbc.Container(id="page-content", className="pt-4"),
    # Show Top 3 Billionaires in the world in cards
    html.H1("The Top 3 Billionaires in the world", style={"text-align":"center","margin-bottom":"2%"}),
    dbc.Row([
        dbc.Col(dbc.Card(card1, color="warning", inverse=True, style={"width":"18rem"}),style={"text-align":"center","margin-left":"10%"}),
        dbc.Col(dbc.Card(card2, color="primary", inverse=True, style={"width":"18rem"}),style={"text-align":"center"}),
        dbc.Col(dbc.Card(card3, color="primary", inverse=True, style={"width":"18rem"}),style={"text-align":"center","margin-right":"10%"})
    ]),
    # Show The Number of Billionaire Born in Each Year
    dbc.Row([
       dbc.Col(html.Div(
           dcc.Graph(figure=px.bar(birthyrs, title="The Number of Billionaires Born in Each Year", labels={'index':'Year','value':'Number'}, color_discrete_sequence= ['#78c2ad']*3, template='simple_white'))
       ))
    ]),
    # Show The Number of Billionaire Born in Each Year
    dbc.Row([
        dbc.Col(html.Div(
            dcc.Graph(id="myGraph")
        ), style={"padding":"2%"})
    ]),

    dropdown,

    dbc.Row([
        dbc.Col(html.Div(
            dcc.Graph(id="myGraph1")
        ), style={"padding":"2%"})
    ]),
    dropdown2,
    #Total Number of Billionaires and Final Worth  of each Category
    dbc.Row([
        dbc.Col(html.Div(
            dcc.Graph(figure= px.treemap(df_cat, path=['category'], values='finalWorth',
                  color='category', hover_data=['finalWorth','N_Billionaires'], title='Total Final Worth of each Category', template='simple_white'))
        ), style={"padding":"2%"}),
        dbc.Col(html.Div(
            dcc.Graph(figure=px.treemap(df_cat, path=['category'], values='N_Billionaires',
                  color='category', hover_data=['finalWorth','N_Billionaires'], title='Total Number of Billionaires of each Category', template='simple_white'))
        ), style={"padding":"2%"})
    ]),
    # Show The percentage of Self Made vs. Non-Self Made Billionaires
    dbc.Row([
        dbc.Col(html.Div(
            dcc.Graph(id="myGraph2")
        ), style={"padding":"2%"})
    ]),
    dropdown3,

    html.H1("About Us", id="aboutus",style={"text-align":"center", "color":"#78c2ad","padding":"2%"}),
    dbc.Row([
        dbc.Col(dbc.Card(hasan,color="warning",inverse=True,style={"width":"18rem"}),style={"text-align":"center","margin-left":"10%"}),
        dbc.Col(dbc.Card(omar,color="warning",inverse=True,style={"width":"18rem"}),style={"text-align":"center","margin-left":"25%","margin-right":"10%"})
    ]),
    html.P("© 2022 Data Visualization DashBoard. all rights deserved | design by Alhasan & Omar ",style={"text-align":"center","padding":"2%"})

])

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [15]:
# Callback of Show The Number of Billionaire Born in Each Year
@app.callback(
    Output("myGraph", "figure"),
    Input("dropMenu", "value")
)
def update_graph(dropdownValue):
    if dropdownValue == "Map":
       fig = px.scatter_geo(iso_df, locations='iso_alpha', projection='orthographic', size='individuals', hover_name='country', color='country', size_max=60, title='The Number of Billionaires in each Country',template='simple_white')
    elif dropdownValue == "BarChart":
        fig = px.bar(iso_df, x='country', y='individuals', labels={'y':'Number'},title="The Number of Billionaire Born in Each Year",  color_discrete_sequence= ['#78c2ad']*3, template='simple_white')
        fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    else:
        fig = px.scatter(iso_df, x='country', y='individuals', size='individuals', size_max=60, hover_name='country', color='country', title='The Number of Billionaires in each Country', template='plotly_white')
        fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    return fig

In [16]:
# Callback of show histogram of age for each country
@app.callback(
    Output("myGraph1", "figure"),
    Input("dropMenu2", "value")
)
def update_graph2(dropdownValue):
    if dropdownValue == "" or dropdownValue is None:
       fig = px.histogram(df, x='age', template='simple_white', color_discrete_sequence= ['#78c2ad']*3, title="The Histogram of Age for each Country")
    else:
       fig = px.histogram(df[df['country'] == dropdownValue], x='age', template='simple_white', color_discrete_sequence= ['#78c2ad']*3, title="The Histogram of Age for "+dropdownValue)
    return fig

In [17]:
# Callback of The percentage of Self Made vs. Non-Self Made Billionaires
@app.callback(
    Output("myGraph2", "figure"),
    Input("dropMenu3", "value")
)
def update_pie(dropdownVal):
    if dropdownVal is None or len(dropdownVal) == 0:
        df_self = pd.DataFrame(df2.groupby("selfMade",as_index=False)["personName"].count())
        df_self.rename(columns={"personName":"num"},inplace=True)
        fig = px.pie(df_self, values="num", names="selfMade",color="selfMade",color_discrete_sequence=["#989898","#78c2ad"] ,hole=0.3, title="The percentage of Self Made vs. Non-Self Made Billionaires",template='simple_white')
    elif dropdownVal[0] == "Female" and len(dropdownVal) ==1 :
        df_tmp = df[df.gender.isin(dropdownVal)]
        df_self = pd.DataFrame(df_tmp.groupby('selfMade', as_index=False)['personName'].count())
        df_self.rename(columns={'personName':'num'}, inplace=True)
        fig = px.pie(df_self, values='num', names='selfMade',color="selfMade",color_discrete_sequence=["#78c2ad","#989898"], hole=.3, title='The percentage of Self Made vs. Non-Self Made Billionaires',template='simple_white')


    else:
        df_tmp = df[df.gender.isin(dropdownVal)]
        df_self = pd.DataFrame(df_tmp.groupby('selfMade', as_index=False)['personName'].count())
        df_self.rename(columns={'personName':'num'}, inplace=True)
        fig = px.pie(df_self, values='num', names='selfMade',color="selfMade",color_discrete_sequence=["#989898","#78c2ad"], hole=.3, title='The percentage of Self Made vs. Non-Self Made Billionaires',template='simple_white')
    return fig

In [ ]:
# Run app Server
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/May/2022 01:45:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /static/images/bernard.jpg HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /static/images/jeff.jpg HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /static/images/elon.jpg HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /static/images/alhasan.jpg HTTP/1.1" 304 -
127.0.0.1 - - [02/May/2022 01:45:02] "GET /static/images/omar.jpg HTTP/1